In [7]:
from agents import *

In [8]:
class Baterias(Thing):
    pass
class Marciano(Thing):
    pass

def program( percepts):
    for p in percepts:
        if isinstance(p, Baterias):
            return 'absorver'
        elif isinstance(p,Marciano):
            return 'registrar'
    return 'ir em frente'

A gente pode incrementar este PEAS colocando-o como 2D. Pra isso basta fazer o ambiente como uma classe descendente de
outra classe ambiente : a classe graphic

In [9]:
class Marte2D(GraphicEnvironment):
    def percept(self, agent):
        '''retorna uma lista de coisas que estao nas proximidades da Sonda'''
        things = self.list_things_at(agent.location)
        return things
    
    def execute_action(self, agent, action):
        '''Altera o estado do Ambiente baseado no que o agente faz.'''
        if action == "ir em frente":
            print('{} resolveu {} na posicao: {}'.format(str(agent)[1:-1], action, agent.location))
            agent.vaiemfrente()
        elif action == "absorver":
            items = self.list_things_at(agent.location, tclass=Baterias)
            if len(items) != 0:
                if agent.absorveu(items[0]): #Sonda absorveu a bateria
                    print('{} absorveu {} na posicao: {}'
                          .format(str(agent)[1:-1], str(items[0])[1:-1], agent.location))
                    self.delete_thing(items[0]) #Remove a bateria absorvida .
        elif action == "registrar":
            items = self.list_things_at(agent.location, tclass=Marciano)
            if len(items) != 0:
                if agent.registrou(items[0]): #Sonda registrou marciano numa posicao
                    print('{} registrou  {} na posicao : {}'
                          .format(str(agent)[1:-1], str(items[0])[1:-1], agent.location))
                    self.delete_thing(items[0]) #Sonda agora pode ignorar este marciano registrado.
    def is_done(self):
        '''Geralmente se para quando o agente morre ( descarrega ), 
        Mas vamos parar quando não houver mais bateria pra pegar nem marcianos a registrar'''
        no_edibles = not any(isinstance(thing, Baterias) or isinstance(thing, Marciano) for thing in self.things)
        dead_agents = not any(agent.is_alive() for agent in self.agents)
        return dead_agents or no_edibles



Mas ao fazermos isso, mudamos o ambiente. Portanto temos que ajustar o Agente para este novo tipo de ambiente. Agora ele deve se deslocar num plano 2D

In [10]:
class SondaMarciana(Agent):
    location = [0,1] # agora um valor 2D
    direction = Direction("down") # pra onde a Sonda esta olhando
    
    def vaiemfrente(self):
        self.location[1] += 1
        
    def absorveu(self, thing):
        '''retorna True se conseguiu absorver'''
        if isinstance(thing, Baterias):
            return True
        return False
    
    def registrou(self, thing):
        ''' retorna True se conseguiu registrar'''
        if isinstance(thing, Marciano):
            return True
        return False
    

Vamos dar uma rodada ...

In [11]:
planicieMarciana = Marte2D(5,20, color={'SondaMarciana': (200,0,0), 'Marciano': (0, 200, 200), 'Baterias': (230, 115, 40)}) # park width is set to 5, and height to 20
Spirit = SondaMarciana(program)
bateria = Baterias()
marciano = Marciano()
planicieMarciana.add_thing(Spirit, [0,1])
planicieMarciana.add_thing(bateria, [0,5])
planicieMarciana.add_thing(marciano, [0,7])
outroMarciano = Marciano()
planicieMarciana.add_thing(outroMarciano, [0,15])
print("Spirit começou em (1,1) voltado para baixo, vamos ver se ele acha a bateria!")
planicieMarciana.run(20)

,,,,
,,,,
,,,,
,,,,
,,,,
,,,,
,,,,
,,,,
,,,,
,,,,
,,,,


Mas nossa sonda, apesar de agora ver um mundo 2D, continua andando só numa direção. Vamos acertar isso :




A sonda vai agora aleatoriamente seguir em frente ou se virar exceto quando encontrar os penhascos que 
 limitam a area explorada ( é eu pus uns penhascos em Marte ). Quando detectar um penhasco ela se vira
    abitrariamente

<table>
    <tr>
        <td><b>Percept:</b> </td>
        <td>Percebe Bateria </td>
        <td>Percebe Marciano</td>
        <td>Percebe Nada</td>
   </tr>
   <tr>
       <td><b>Ação</b> </td>
       <td>Absorve</td>
       <td>Registra</td>
       <td>
       <table>
           <tr>
               <td><b>Registra que está na beira do penhasco</b></td>
               <td>Tem penhasco</td>
               <td>Não tem penhasco</td>
           </tr>
           <tr>
               <td><b>Ação:</b></td>
               <td>Vira Esquerda / Vira Direita <br> ( 50% - 50% chance )</td>
               <td>Vira Esquerda / Vira Direita / Segue em Frente <br> ( 25% - 25% - 50% chance )</td>
           </tr>
       </table>
       </td>
   </tr>
        
</table>

In [12]:
from random import choice

class SondaMarciana(Agent):
    location = [0,1]
    direction = Direction("down")
    
    def vaiemfrente(self, success=True):
        '''vai em frente se tem uma destinacao valida '''
        if not success:
            return
        if self.direction.direction == Direction.R:
            self.location[0] += 1
        elif self.direction.direction == Direction.L:
            self.location[0] -= 1
        elif self.direction.direction == Direction.D:
            self.location[1] += 1
        elif self.direction.direction == Direction.U:
            self.location[1] -= 1
    
    def virou(self, d):
        self.direction = self.direction + d
        
    def absorveu(self, thing):
        if isinstance(thing, Baterias):
            return True
        return False
    
    def registrou(self, thing):
        if isinstance(thing, Marciano):
            return True
        return False
        
def program(percepts):
    '''retorna uma ação baseada numa percepcao'''
        
    for p in percepts: 
        if isinstance(p, Baterias):
            return 'absorver'
        elif isinstance(p, Marciano):
            return 'registrar'
        if isinstance(p,Bump): # checa se é um penhasco
            turn = False
            choice = random.choice((1,2));
        else:
            choice = random.choice((1,2,3,4)) # 1-direita, 2-esquerda, ou segue em frente
    if choice == 1:
        return 'viradireita'
    elif choice == 2:
        return 'viraesquerda'
    else:
        return 'vaiemfrente'
    

Bom fizemos o agente com novas habilidades. Agora temos que acertar o ambiente 2D para que ele se altere como resultado 
das ações do agente 

In [13]:
class Marte2D(GraphicEnvironment):
    def percept(self, agent):
        things = self.list_things_at(agent.location)
        loc = copy.deepcopy(agent.location) 
        #verifica se a Sonda está indo pro penhasco
        if agent.direction.direction == Direction.R:
            loc[0] += 1
        elif agent.direction.direction == Direction.L:
            loc[0] -= 1
        elif agent.direction.direction == Direction.D:
            loc[1] += 1
        elif agent.direction.direction == Direction.U:
            loc[1] -= 1
        if not self.is_inbounds(loc):
            things.append(Bump())
        return things
    
    def execute_action(self, agent, action):
        if action == 'viradireita':
            print('{} resolveu {} na posicao: {}'.format(str(agent)[1:-1], action, agent.location))
            agent.virou(Direction.R)
        elif action == 'viraesquerda':
            print('{} resolveu {} na posicao: {}'.format(str(agent)[1:-1], action, agent.location))
            agent.virou(Direction.L)
        elif action == 'vaiemfrente':
            print('{} resolveu andar {}  na posicao: {}'.format(str(agent)[1:-1], agent.direction.direction, agent.location))
            agent.vaiemfrente()
        elif action == "absorver":
            items = self.list_things_at(agent.location, tclass=Baterias)
            if len(items) != 0:
                if agent.absorveu(items[0]):
                    print('{} absorveu {} na posicao: {}'
                          .format(str(agent)[1:-1], str(items[0])[1:-1], agent.location))
                    self.delete_thing(items[0])
        elif action == "registrar":
            items = self.list_things_at(agent.location, tclass=Marciano)
            if len(items) != 0:
                if agent.registrou(items[0]):
                    print('{} registrou {} na posicao: {}'
                          .format(str(agent)[1:-1], str(items[0])[1:-1], agent.location))
                    self.delete_thing(items[0])
                    
    def is_done(self):

        no_edibles = not any(isinstance(thing, Baterias) or isinstance(thing, Marciano) for thing in self.things)
        dead_agents = not any(agent.is_alive() for agent in self.agents)
        return dead_agents or no_edibles


In [15]:
planicieM = Marte2D(10,10, color={'SondaMarciana': (200,0,0), 'Marciano': (0, 200, 200), 'Baterias': (230, 115, 40)})
Spirit = SondaMarciana(program)
bateria1 = Baterias()
marciano1 = Marciano()
planicieM.add_thing(Spirit, [0,0])
planicieM.add_thing(bateria1, [1,2])
planicieM.add_thing(marciano1, [0,1])

bateria2 = Baterias()
marciano2 = Marciano()
planicieM.add_thing(bateria2, [4,3])
planicieM.add_thing(marciano2, [2,4])

print("Spirit inicia em [0,0], vira para o Sul.")
planicieM.run(120)

,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
